In [3]:
from PyPDF2 import PdfReader
import chromadb

# Initialize ChromaDB client
client = chromadb.Client()

# Create or get the collection
collection = client.get_or_create_collection(name="resume_collection")

# List of PDF files
pdf_files = ["Kartik_Shingate_Resume.pdf", "Alice_Smith_Resume.pdf", "Bob_Johnson_Resume.pdf", "Charlie_Davis_Resume.pdf"]

# Process each PDF file
for pdf_file in pdf_files:
    reader = PdfReader(pdf_file)
    text = ""
    
    # Extract text from all pages
    for page in reader.pages:
        text += page.extract_text()
    
    # Use the file name (without extension) as the ID
    person_id = pdf_file.replace(".pdf", "").replace("_", " ")
    
    # Add document and ID to the collection
    collection.add(
        documents=[text.strip()],
        ids=[person_id]
    )

print("Documents successfully added to ChromaDB!")


Documents successfully added to ChromaDB!


In [4]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0.5,
    timeout=None,
    max_retries=2,
    api_key='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'
    # other params...
)

In [18]:
job_post = '''
Swiggy is looking for a talented and results-driven Data Scientist to join our team in Bangalore (with hybrid/remote options available). 
In this role, you will develop and deploy machine learning models to optimize delivery times, enhance user experiences, and drive operational efficiency.
You will analyze large datasets to uncover trends and actionable insights, collaborate with cross-functional teams to design data-driven solutions, 
and build recommendation systems and predictive analytics models. We’re seeking candidates with 2-5 years of experience in data science, proficiency in 
Python, R, or Scala, and familiarity with machine learning frameworks like TensorFlow or Scikit-learn. Hands-on experience with SQL and big data tools
(e.g., Hadoop, Spark) is essential, alongside a strong understanding of statistical methods and data visualization techniques.Preferred qualifications 
include expertise in deep learning, NLP, recommendation systems, and A/B testing, with a bonus for prior experience in e-commerce or logistics domains.
Join Swiggy to work on impactful, large-scale projects and help redefine how India eats!
'''

In [20]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You have to extract following details from provided job post. Extract JOB_ROLE, EXPERIENCE, SKILLS in json format  extract only the curly bracket information. Dont provide preamble information",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
result = chain.invoke(
    {
        "input": job_post,
    }
)

print(result.content)

{ 
"JOB_ROLE": "Data Scientist", 
"EXPERIENCE": "2-5 years", 
"SKILLS": ["Python", "R", "Scala", "TensorFlow", "Scikit-learn", "SQL", "Hadoop", "Spark", "Machine Learning", "Deep Learning", "NLP", "Data Visualization"] 
}


In [22]:
import json

In [24]:
Details = json.loads(result.content)

In [26]:
Details

{'JOB_ROLE': 'Data Scientist',
 'EXPERIENCE': '2-5 years',
 'SKILLS': ['Python',
  'R',
  'Scala',
  'TensorFlow',
  'Scikit-learn',
  'SQL',
  'Hadoop',
  'Spark',
  'Machine Learning',
  'Deep Learning',
  'NLP',
  'Data Visualization']}

In [28]:
SE = ' '.join([' '.join(Details['SKILLS']), ' '.join(Details['EXPERIENCE'])])
print(SE)

Python R Scala TensorFlow Scikit-learn SQL Hadoop Spark Machine Learning Deep Learning NLP Data Visualization 2 - 5   y e a r s


In [30]:
results = collection.query(
    query_texts=[SE], # Chroma will embed this for you
    n_results=1 # how many results to return
)
print(results)

{'ids': [['Charlie Davis Resume']], 'embeddings': None, 'documents': [['Resume\nStudent 3: Charlie Davis\nContact Information\nEmail: charlie.davis@example.com\nPhone: +1 456 789 0123\nLinkedIn: linkedin.com/in/charliedavis\nProfessional Summary\nRecent graduate in Data Science with hands-on experience in machine learning projects and big data technologies.\nSkills\nPython, Hadoop, Spark, SQL, Machine Learning, Deep Learning, Data Engineering, Big Data Analytics\nExperience\nMachine Learning Intern, JKL Analytics (2023)\n- Built a machine learning model to predict customer churn with 85% accuracy.\n- Deployed the model as an API using Flask.\nResearch Assistant, University Data Lab (2021 - 2022)\n- Conducted research on natural language processing techniques.\n- Published findings in a peer-reviewed journal.']], 'uris': None, 'data': None, 'metadatas': [[None]], 'distances': [[1.001784324645996]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, 

In [32]:
names = results.get('ids', [])
print(names)

[['Charlie Davis Resume']]


In [34]:
portfolios = ''.join(results['documents'][0])
portfolios

'Resume\nStudent 3: Charlie Davis\nContact Information\nEmail: charlie.davis@example.com\nPhone: +1 456 789 0123\nLinkedIn: linkedin.com/in/charliedavis\nProfessional Summary\nRecent graduate in Data Science with hands-on experience in machine learning projects and big data technologies.\nSkills\nPython, Hadoop, Spark, SQL, Machine Learning, Deep Learning, Data Engineering, Big Data Analytics\nExperience\nMachine Learning Intern, JKL Analytics (2023)\n- Built a machine learning model to predict customer churn with 85% accuracy.\n- Deployed the model as an API using Flask.\nResearch Assistant, University Data Lab (2021 - 2022)\n- Conducted research on natural language processing techniques.\n- Published findings in a peer-reviewed journal.'

In [36]:
#from langchain_core.prompts import ChatPromptTemplate

email_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            '''We are working as a placement officer in placement consultancy codespyder.
            We have short listed candidates portfolio as per the requrement of TCS job post.
            Create an email to recruitment team of TCS mentioning that we have best suitable candidates for your job post.
            Do not provide preamble''',
        ),
        ("human", "{input}"),
    ]
)

email_chain = email_prompt | llm
result = email_chain.invoke(
    {
        "input": portfolios,
    }
)

print(result.content)

Subject: Shortlisted Candidates for TCS Job Post

Dear TCS Recruitment Team,

We are excited to inform you that we have shortlisted a highly skilled candidate, Charlie Davis, whose portfolio aligns with the requirements of your job post. With a strong background in Data Science and hands-on experience in machine learning projects, Charlie is an ideal fit for your team.

Charlie's skills include:

* Programming languages: Python
* Big Data technologies: Hadoop, Spark
* Data Analysis: SQL, Machine Learning, Deep Learning, Data Engineering, Big Data Analytics

Some of Charlie's notable experiences include:

* Building a machine learning model to predict customer churn with 85% accuracy as a Machine Learning Intern at JKL Analytics
* Deploying the model as an API using Flask
* Conducting research on natural language processing techniques as a Research Assistant at University Data Lab
* Publishing findings in a peer-reviewed journal

We believe Charlie would be a valuable asset to your team